In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install -q langchain langchain-community langchain-groq sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 55.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.5/443.5 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.1 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/5

In [2]:
import os
import json
import numpy as np
import pandas as pd
from typing import List, Dict, Any, Optional
import faiss
from sentence_transformers import SentenceTransformer
import re
from groq import Groq
import logging
from dataclasses import dataclass
from datetime import datetime
import pickle
import gradio as gr

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Configuration
@dataclass
class Config:
    GROQ_API_KEY: str = "your_groq_api_key_here"  # Replace with your actual API key
    EMBEDDING_MODEL: str = "all-MiniLM-L6-v2"
    LLM_MODEL: str = "llama3-8b-8192"  # Updated to working model
    VECTOR_DIM: int = 384
    CHUNK_SIZE: int = 1000
    CHUNK_OVERLAP: int = 200
    DATA_PATH: str = "/kaggle/input/"

class DocumentProcessor:
    """Handles document preprocessing and chunking"""
    
    def __init__(self, chunk_size: int = 1000, chunk_overlap: int = 200):
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
    
    def load_documents(self, file_path: str) -> str:
        """Load documents from text file"""
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()
            logger.info(f"Loaded document with {len(content)} characters")
            return content
        except Exception as e:
            logger.error(f"Error loading document: {e}")
            return ""
    
    def clean_text(self, text: str) -> str:
        """Clean and normalize text"""
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'\n+', '\n', text)
        return text.strip()
    
    def chunk_text(self, text: str) -> List[Dict[str, Any]]:
        """Split text into overlapping chunks with metadata"""
        chunks = []
        words = text.split()
        
        for i in range(0, len(words), self.chunk_size - self.chunk_overlap):
            chunk_words = words[i:i + self.chunk_size]
            chunk_text = ' '.join(chunk_words)
            
            chunks.append({
                'text': chunk_text,
                'chunk_id': len(chunks),
                'start_word': i,
                'end_word': min(i + self.chunk_size, len(words)),
                'word_count': len(chunk_words)
            })
        
        logger.info(f"Created {len(chunks)} chunks")
        return chunks

class VectorStore:
    """Handles vector embeddings and similarity search using FAISS"""
    
    def __init__(self, embedding_model: str = "all-MiniLM-L6-v2", vector_dim: int = 384):
        self.model = SentenceTransformer(embedding_model)
        self.vector_dim = vector_dim
        # Use L2 distance instead of inner product for more stable results
        self.index = faiss.IndexFlatL2(vector_dim)
        self.documents = []
        self.embeddings = []
    
    def add_documents(self, chunks: List[Dict[str, Any]]):
        """Add documents to vector store"""
        texts = [chunk['text'] for chunk in chunks]
        embeddings = self.model.encode(texts, normalize_embeddings=True)
        
        # Add to FAISS index
        self.index.add(embeddings.astype('float32'))
        
        # Store documents and embeddings
        self.documents.extend(chunks)
        self.embeddings.extend(embeddings)
        
        logger.info(f"Added {len(chunks)} documents to vector store")
    
    def search(self, query: str, k: int = 5) -> List[Dict[str, Any]]:
        """Search for similar documents"""
        if len(self.documents) == 0:
            return []
            
        query_embedding = self.model.encode([query], normalize_embeddings=True)
        
        # Search in FAISS index - using L2 distance (lower is better)
        distances, indices = self.index.search(query_embedding.astype('float32'), min(k, len(self.documents)))
        
        results = []
        for distance, idx in zip(distances[0], indices[0]):
            if idx >= 0 and idx < len(self.documents):  # Valid index check
                result = self.documents[idx].copy()
                # Convert L2 distance to similarity score (higher is better)
                similarity_score = 1.0 / (1.0 + float(distance))
                result['similarity_score'] = similarity_score
                results.append(result)
        
        # Sort by similarity score (highest first)
        results.sort(key=lambda x: x['similarity_score'], reverse=True)
        return results

class LegalReasoner:
    """Handles legal reasoning using LLM"""
    
    def __init__(self, api_key: str, model: str = "llama3-8b-8192"):
        self.client = Groq(api_key=api_key)
        self.model = model
    
    def analyze_case(self, case_description: str, relevant_precedents: List[Dict[str, Any]]) -> Dict[str, Any]:
        """Analyze legal case with precedents"""
        
        precedent_context = "\n\n".join([
            f"Precedent {i+1} (Relevance: {doc['similarity_score']:.3f}):\n{doc['text'][:500]}"
            for i, doc in enumerate(relevant_precedents[:3])  # Use top 3 to avoid token limits
        ])
        
        system_prompt = """You are an expert legal AI assistant specializing in Indian constitutional law and privacy rights. 
        Analyze legal cases by examining precedents and applying legal reasoning principles. 
        Provide balanced analysis for both sides of the argument. Keep responses concise and focused."""
        
        user_prompt = f"""
        Case Analysis Request:
        
        Current Case: {case_description[:1000]}
        
        Relevant Legal Precedents:
        {precedent_context}
        
        Please provide a comprehensive legal analysis including:
        1. Case Summary (2-3 sentences)
        2. Key Legal Issues (3-4 main issues)
        3. Applicable Precedents and their relevance
        4. Arguments for the Petitioner (4-5 key points)
        5. Arguments for the Government/Respondent (4-5 key points)
        6. Likely Verdict and Reasoning (brief analysis)
        7. Legal Implications (2-3 key implications)
        
        Format your response clearly with headings.
        """
        
        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                max_tokens=3000,
                temperature=0.3
            )
            
            return {
                'analysis': response.choices[0].message.content,
                'model_used': self.model,
                'timestamp': datetime.now().isoformat()
            }
        
        except Exception as e:
            logger.error(f"Error in legal analysis: {e}")
            return {
                'analysis': f'Error in analysis: {str(e)}\n\nPlease check your API key or try again.',
                'model_used': self.model,
                'timestamp': datetime.now().isoformat()
            }
    
    def generate_arguments(self, case_description: str, side: str, precedents: List[Dict[str, Any]]) -> str:
        """Generate specific arguments for petitioner or respondent"""
        
        precedent_context = "\n\n".join([
            f"Precedent {i+1}:\n{doc['text'][:400]}"
            for i, doc in enumerate(precedents[:2])  # Use top 2 to avoid token limits
        ])
        
        system_prompt = f"""You are a legal expert preparing arguments for the {side} in an Indian constitutional law case. 
        Use relevant precedents and legal principles to build a strong argument. Be concise and focused."""
        
        user_prompt = f"""
        Case: {case_description[:800]}
        
        Relevant Precedents:
        {precedent_context}
        
        Prepare detailed legal arguments for the {side}. Include:
        1. Legal grounds and constitutional provisions (2-3 main grounds)
        2. Precedent citations and their application (2-3 cases)
        3. Policy considerations (2-3 points)
        4. Potential counterarguments and responses (2-3 responses)
        
        Be specific and cite relevant case law and constitutional articles.
        """
        
        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                max_tokens=1500,
                temperature=0.4
            )
            
            return response.choices[0].message.content
            
        except Exception as e:
            logger.error(f"Error generating arguments: {e}")
            return f"Error generating arguments: {str(e)}\n\nPlease check your API key or try again."

class LegalAgent:
    """Main agentic system orchestrating the legal analysis workflow"""
    
    def __init__(self, config: Config):
        self.config = config
        self.document_processor = DocumentProcessor(config.CHUNK_SIZE, config.CHUNK_OVERLAP)
        self.vector_store = VectorStore(config.EMBEDDING_MODEL, config.VECTOR_DIM)
        self.legal_reasoner = LegalReasoner(config.GROQ_API_KEY, config.LLM_MODEL)
        self.knowledge_base_ready = False
    
    def build_knowledge_base(self, file_path: str = None, text_content: str = None):
        """Build the legal knowledge base from documents or text"""
        logger.info("Building knowledge base...")
        
        if text_content and text_content.strip():
            raw_text = text_content
            logger.info(f"Using provided text content ({len(text_content)} characters)")
        elif file_path and os.path.exists(file_path):
            raw_text = self.document_processor.load_documents(file_path)
            logger.info(f"Loaded from file: {file_path}")
        else:
            # Default sample legal text focused on Puttaswamy principles
            raw_text = """The Constitution of India, Article 21 states that "No person shall be deprived of his life or personal liberty except according to procedure established by law." The Supreme Court in K.S. Puttaswamy v. Union of India (2017) unanimously declared that the right to privacy is a fundamental right under the Indian Constitution. The Court held that privacy is not an absolute right and can be restricted by the State, but any such restriction must satisfy the three-fold test: (i) legality - there must be a law; (ii) legitimate state aim - the law must seek to achieve a legitimate state aim; and (iii) proportionality - there must be a rational nexus between the objects and the means adopted. The Court further held that privacy includes at its core the preservation of personal intimacies, the sanctity of family life, marriage, procreation, the home and sexual orientation. Privacy also connotes a right to be left alone. The security of one's privacy against arbitrary intrusion by the police is basic to a free society. The right to privacy is protected as an intrinsic part of the right to life and personal liberty under Article 21 and as a part of the freedoms guaranteed by Part III of the Constitution. Privacy includes at its core the preservation of personal intimacies, the sanctity of family life, marriage, procreation, the home and sexual orientation. Privacy also connotes a right to be left alone. Privacy safeguards individual autonomy and recognises the ability of the individual to control vital aspects of his or her life. Personal autonomy includes the ability of an individual to make choices governing his or her life. Informational self-determination is a facet of the right to privacy. The development of technology and the growth of the means of mass communication along with the legitimate expectation that information which an individual has shared in confidence will not be shared with others, mandates recognition of the fundamental right to privacy. The Fundamental Rights guaranteed under Part III of the Constitution are natural rights inherent in the status of a citizen. Aadhaar enables the creation of a profile of an individual and serves the basis of a surveillance state. The biometric and demographic information of the individual is shared with requesting entities. There is a likelihood of commercial exploitation of biometric and demographic information by the private entities. The law must ensure the presence of safeguards against such misuse."""
            logger.info("Using default Puttaswamy-based legal text")
        
        if not raw_text:
            return False, "Failed to load documents - no content available"
        
        # Clean and chunk the text
        clean_text = self.document_processor.clean_text(raw_text)
        chunks = self.document_processor.chunk_text(clean_text)
        
        if not chunks:
            return False, "Failed to create text chunks"
        
        # Add to vector store
        try:
            self.vector_store.add_documents(chunks)
            self.knowledge_base_ready = True
            logger.info("Knowledge base built successfully")
            return True, f"Knowledge base built successfully with {len(chunks)} chunks"
        except Exception as e:
            logger.error(f"Error adding documents to vector store: {e}")
            return False, f"Error building knowledge base: {str(e)}"
    
    def query_case(self, case_description: str, num_precedents: int = 5) -> Dict[str, Any]:
        """Main method to analyze a legal case"""
        if not self.knowledge_base_ready:
            return {"error": "Knowledge base not ready. Please build it first."}
        
        logger.info(f"Processing case query: {case_description[:100]}...")
        
        # Retrieve relevant precedents
        relevant_precedents = self.vector_store.search(case_description, k=num_precedents)
        
        # Perform legal analysis
        analysis = self.legal_reasoner.analyze_case(case_description, relevant_precedents)
        
        # Generate arguments for both sides
        petitioner_args = self.legal_reasoner.generate_arguments(
            case_description, "Petitioner", relevant_precedents
        )
        
        respondent_args = self.legal_reasoner.generate_arguments(
            case_description, "Government/Respondent", relevant_precedents
        )
        
        result = {
            'case_description': case_description,
            'relevant_precedents': relevant_precedents,
            'legal_analysis': analysis,
            'petitioner_arguments': petitioner_args,
            'respondent_arguments': respondent_args,
            'processing_timestamp': datetime.now().isoformat()
        }
        
        return result

# Initialize global agent
config = Config()
agent = LegalAgent(config)

# Gradio Interface Functions
def setup_knowledge_base(api_key, legal_text=None):
    """Setup the knowledge base"""
    if not api_key.strip():
        return "❌ Please provide a valid Groq API key", ""
    
    # Update the API key
    config.GROQ_API_KEY = api_key
    agent.legal_reasoner = LegalReasoner(api_key, config.LLM_MODEL)
    
    # Hardcoded path to the Puttaswamy judgment file
    puttaswamy_file_path = "/kaggle/input/datasethannoworks/puttaswamy_judgment.txt"
    
    # Build knowledge base
    if legal_text and legal_text.strip():
        # Use provided text
        success, message = agent.build_knowledge_base(text_content=legal_text)
        source_info = "Custom legal text provided"
    elif os.path.exists(puttaswamy_file_path):
        # Use the hardcoded Puttaswamy file
        success, message = agent.build_knowledge_base(file_path=puttaswamy_file_path)
        source_info = f"Loaded from: {puttaswamy_file_path}"
    else:
        # Use default legal text if file not found
        success, message = agent.build_knowledge_base()
        source_info = f"Default privacy law precedents (File not found at: {puttaswamy_file_path})"
    
    if success:
        status = f"✅ {message}"
        kb_info = f"Knowledge Base Status: Ready\nChunks: {len(agent.vector_store.documents)}\nEmbedding Model: {config.EMBEDDING_MODEL}\nSource: {source_info}"
    else:
        status = f"❌ {message}"
        kb_info = "Knowledge Base Status: Not Ready"
    
    return status, kb_info

def analyze_legal_case(case_description, num_precedents):
    """Analyze a legal case"""
    if not agent.knowledge_base_ready:
        return "❌ Knowledge base not ready. Please set up the knowledge base first.", "", "", "", ""
    
    if not case_description.strip():
        return "❌ Please provide a case description.", "", "", "", ""
    
    try:
        # Analyze the case
        result = agent.query_case(case_description, num_precedents)
        
        if "error" in result:
            return f"❌ {result['error']}", "", "", "", ""
        
        # Format precedents
        precedents_text = ""
        for i, precedent in enumerate(result['relevant_precedents'], 1):
            precedents_text += f"**Precedent {i}** (Similarity: {precedent['similarity_score']:.3f})\n"
            precedents_text += f"{precedent['text'][:400]}...\n\n"
        
        # Format analysis
        analysis_text = result['legal_analysis']['analysis']
        
        # Format arguments
        petitioner_text = result['petitioner_arguments']
        respondent_text = result['respondent_arguments']
        
        return "✅ Analysis completed successfully!", precedents_text, analysis_text, petitioner_text, respondent_text
        
    except Exception as e:
        return f"❌ Error during analysis: {str(e)}", "", "", "", ""

def search_precedents(query, num_results):
    """Search for relevant precedents"""
    if not agent.knowledge_base_ready:
        return "❌ Knowledge base not ready. Please set up the knowledge base first."
    
    if not query.strip():
        return "❌ Please provide a search query."
    
    try:
        results = agent.vector_store.search(query, k=num_results)
        
        search_results = f"**Search Results for:** {query}\n\n"
        for i, result in enumerate(results, 1):
            search_results += f"**Result {i}** (Similarity: {result['similarity_score']:.3f})\n"
            search_results += f"{result['text'][:300]}...\n\n"
        
        return search_results
        
    except Exception as e:
        return f"❌ Error during search: {str(e)}"

# Default case for demonstration
DEFAULT_CASE = """In 2025, a new case challenges a government policy that mandates all citizens to submit biometric data for accessing public services. A petitioner argues that the policy violates their privacy rights, citing the 2017 Supreme Court judgment that ruled the Right to Privacy as a fundamental right under Article 21 of the Indian Constitution.

The government argues that the biometric data collection is necessary for:
1. Preventing fraud in public service delivery
2. Ensuring efficient distribution of benefits
3. National security considerations
4. Digital India initiative compliance

The petitioner contends that:
1. Mandatory biometric collection violates fundamental right to privacy
2. No adequate safeguards for data protection
3. Disproportionate state intrusion without compelling state interest
4. Lack of informed consent and opt-out mechanisms"""

# Create Gradio Interface
def create_gradio_interface():
    """Create the Gradio interface"""
    
    with gr.Blocks(title="Legal Intelligence Agentic System", theme=gr.themes.Soft()) as interface:
        
        gr.HTML("""
        <div style="text-align: center; padding: 20px;">
            <h1>⚖️ Legal Intelligence Agentic System</h1>
            <p>AI-powered legal case analysis with precedent-based reasoning</p>
            <p><em>Hannoworks Internship Assignment Implementation</em></p>
        </div>
        """)
        
        with gr.Tab("🔧 Setup"):
            gr.Markdown("### Configure your Legal Intelligence System")
            
            with gr.Row():
                with gr.Column():
                    api_key_input = gr.Textbox(
                        label="Groq API Key",
                        placeholder="Enter your Groq API key here...",
                        type="password",
                        info="Get your free API key from https://console.groq.com/"
                    )
                    
                    legal_text_input = gr.Textbox(
                        label="Legal Knowledge Base (Optional)",
                        placeholder="Paste your legal documents here or leave empty to use default...",
                        lines=10,
                        info="Add your own legal texts or use the default privacy law precedents"
                    )
                    
                    setup_btn = gr.Button("🔨 Build Knowledge Base", variant="primary")
                
                with gr.Column():
                    setup_status = gr.Textbox(label="Setup Status", interactive=False)
                    kb_info = gr.Textbox(label="Knowledge Base Info", lines=5, interactive=False)
            
            setup_btn.click(
                fn=setup_knowledge_base,
                inputs=[api_key_input, legal_text_input],
                outputs=[setup_status, kb_info]
            )
        
        with gr.Tab("📋 Case Analysis"):
            gr.Markdown("### Analyze Legal Cases with AI-powered Reasoning")
            
            with gr.Row():
                with gr.Column(scale=2):
                    case_input = gr.Textbox(
                        label="Case Description",
                        placeholder="Describe the legal case you want to analyze...",
                        lines=8,
                        value=DEFAULT_CASE,
                        info="Provide detailed case facts, legal issues, and parties' positions"
                    )
                    
                    with gr.Row():
                        precedents_slider = gr.Slider(
                            minimum=3,
                            maximum=10,
                            value=5,
                            step=1,
                            label="Number of Precedents to Retrieve"
                        )
                        
                        analyze_btn = gr.Button("⚖️ Analyze Case", variant="primary")
                
                with gr.Column(scale=1):
                    analysis_status = gr.Textbox(label="Analysis Status", interactive=False)
            
            with gr.Row():
                with gr.Column():
                    precedents_output = gr.Textbox(
                        label="📚 Relevant Precedents",
                        lines=10,
                        interactive=False
                    )
                
                with gr.Column():
                    analysis_output = gr.Textbox(
                        label="🔍 Legal Analysis",
                        lines=10,
                        interactive=False
                    )
            
            with gr.Row():
                with gr.Column():
                    petitioner_output = gr.Textbox(
                        label="👤 Petitioner Arguments",
                        lines=8,
                        interactive=False
                    )
                
                with gr.Column():
                    respondent_output = gr.Textbox(
                        label="🏛️ Government/Respondent Arguments",
                        lines=8,
                        interactive=False
                    )
            
            analyze_btn.click(
                fn=analyze_legal_case,
                inputs=[case_input, precedents_slider],
                outputs=[analysis_status, precedents_output, analysis_output, petitioner_output, respondent_output]
            )
        
        with gr.Tab("🔍 Search Precedents"):
            gr.Markdown("### Search Legal Precedents and Case Law")
            
            with gr.Row():
                with gr.Column():
                    search_query = gr.Textbox(
                        label="Search Query",
                        placeholder="Enter legal concepts, case names, or constitutional provisions...",
                        info="e.g., 'right to privacy Article 21', 'biometric data collection', 'fundamental rights'"
                    )
                    
                    search_results_slider = gr.Slider(
                        minimum=1,
                        maximum=10,
                        value=5,
                        step=1,
                        label="Number of Results"
                    )
                    
                    search_btn = gr.Button("🔍 Search", variant="primary")
                
                with gr.Column():
                    search_output = gr.Textbox(
                        label="Search Results",
                        lines=15,
                        interactive=False
                    )
            
            search_btn.click(
                fn=search_precedents,
                inputs=[search_query, search_results_slider],
                outputs=[search_output]
            )
        
        with gr.Tab("ℹ️ About"):
            gr.Markdown("""
            ## About Legal Intelligence Agentic System
            
            This system demonstrates an AI-powered legal intelligence platform that can:
            
            ### 🎯 Key Features:
            - **Precedent-based Analysis**: Retrieves and analyzes relevant legal precedents
            - **Multi-perspective Arguments**: Generates arguments for both petitioner and respondent
            - **Constitutional Law Focus**: Specialized in Indian constitutional law and privacy rights
            - **RAG Pipeline**: Uses Retrieval-Augmented Generation for accurate legal reasoning
            - **Semantic Search**: Advanced vector similarity search for legal documents
            
            ### 🔧 Technical Architecture:
            - **Document Processing**: Text chunking and cleaning
            - **Vector Store**: FAISS-powered similarity search with Sentence Transformers
            - **Legal Reasoning**: Groq LLM with specialized legal prompts
            - **Agentic Workflow**: Multi-step reasoning pipeline
            
            ### 📚 Components:
            1. **DocumentProcessor**: Handles legal text preprocessing
            2. **VectorStore**: Manages embeddings and similarity search
            3. **LegalReasoner**: Provides AI-powered legal analysis
            4. **LegalAgent**: Orchestrates the complete workflow
            
            ### 🚀 Usage:
            1. **Setup**: Enter your Groq API key and optionally add legal texts
            2. **Analyze**: Input case descriptions for comprehensive analysis
            3. **Search**: Find relevant precedents and case law
            
            ---
            *Developed for Hannoworks Internship Assignment*
            """)
    
    return interface

# Launch the interface
if __name__ == "__main__":
    # Create and launch the interface
    interface = create_gradio_interface()
    
    # For Kaggle, use share=True to create a public link
    interface.launch(
        share=True,  # Creates a public link for Kaggle
        server_name="0.0.0.0",  # Allows external access
        server_port=7860,  # Default Gradio port
        show_error=True,  # Show detailed errors
        debug=True  # Enable debug mode
    )

2025-08-10 20:39:26.484689: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754858366.736151      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754858366.806024      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Keyboard interruption in main thread... closing server.
Killing tunnel 0.0.0.0:7860 <> https://c5bfc0c3cda5528dc6.gradio.live
